In [2]:
# Beginning by reading in the relevant libraries
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

# And the Stack Overflow 2020 survey results
df = pd.read_csv('./data/2020/survey_results_public.csv')
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [5]:
# We're wondering if gender contributes to salary, so let's look at that first
study = df['Gender'].value_counts().reset_index()
study.head()

,index,Gender
0,Man,46013
1,Woman,3844
2,"Non-binary, genderqueer, or gender non-conforming",385
3,"Man;Non-binary, genderqueer, or gender non-con...",121
4,"Woman;Non-binary, genderqueer, or gender non-c...",92


In [9]:
# Now let's check out the percentages
study['perc'] = study['Gender']/np.sum(study['Gender'])
study

,index,Gender,perc
0,Man,46013,0.910121
1,Woman,3844,0.076033
2,"Non-binary, genderqueer, or gender non-conforming",385,0.007615
3,"Man;Non-binary, genderqueer, or gender non-con...",121,0.002393
4,"Woman;Non-binary, genderqueer, or gender non-c...",92,0.001820
5,Woman;Man,76,0.001503
6,"Woman;Man;Non-binary, genderqueer, or gender n...",26,0.000514


In [12]:
# Now let's look at salary for these different genders, using mean_amt from Lesson 1
possible_vals=["Man","Woman","Non-binary, genderqueer, or gender non-conforming"]

def mean_amt(df, col_name, col_mean, look_for):
    '''
    INPUT:
    df - the pandas dataframe you want to search
    col_name - the column name you want to look through
    col_count - the column you want to count values from
    col_mean - the column you want the mean amount for
    look_for - a list of strings you want to search for in each row of df[col]
    
    OUTPUT:
    df_all - holds sum, square, total, mean, variance, and standard deviation for the col_mean
    '''
    new_df = defaultdict(int)
    squares_df = defaultdict(int)
    denoms = dict()
    for val in look_for:
        denoms[val] = 0
        for idx in range(df.shape[0]):
            if df[col_name].isnull()[idx] == False:
                if val in df[col_name][idx] and df[col_mean][idx] > 0:
                    new_df[val] += df[col_mean][idx]
                    squares_df[val] += df[col_mean][idx]**2 #Needed to understand the spread
                    denoms[val] += 1 
    
    # Turn into dataframes
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    squares_df = pd.DataFrame(pd.Series(squares_df)).reset_index()
    denoms = pd.DataFrame(pd.Series(denoms)).reset_index()
    
    # Change the column names
    new_df.columns = [col_name, 'col_sum']
    squares_df.columns = [col_name, 'col_squares']
    denoms.columns = [col_name, 'col_total']
    
    # Merge dataframes
    df_means = pd.merge(new_df, denoms)
    df_all = pd.merge(df_means, squares_df)
    
    # Additional columns needed for analysis
    df_all['mean_col'] = df_means['col_sum']/df_means['col_total']
    df_all['var_col'] = df_all['col_squares']/df_all['col_total'] - df_all['mean_col']**2
    df_all['std_col'] = np.sqrt(df_all['var_col'])
    df_all['lower_95'] = df_all['mean_col'] - 1.96*df_all['std_col']/np.sqrt(df_all['col_total'])
    df_all['upper_95'] = df_all['mean_col'] + 1.96*df_all['std_col']/np.sqrt(df_all['col_total'])
    return df_all



df_all = mean_amt(df, 'Gender', 'ConvertedComp', possible_vals)

df_all.sort_values('mean_col', ascending=False)

,Gender,col_sum,col_total,col_squares,mean_col,var_col,std_col,lower_95,upper_95
2,"Non-binary, genderqueer, or gender non-conforming",4.285885e+07,387,1.835521e+13,110746.379845,3.516471e+10,187522.570772,92063.056199,129429.703491
0,Man,3.061857e+09,29458,1.821223e+15,103939.733383,5.102092e+10,225878.100250,101360.274742,106519.192025
1,Woman,2.330328e+08,2352,1.301546e+14,99078.585884,4.552125e+10,213357.105342,90455.861130,107701.310639


In [13]:
# This provides an answer to our first question.  Is this influenced by country?  Let's look at this data on a country-by-country basis.
study_country = df['Country'].value_counts().reset_index()
study_country.head()

,index,Country
0,United States,12469
1,India,8403
2,United Kingdom,3896
3,Germany,3890
4,Canada,2191


In [14]:
df.groupby('Country', as_index=False)['ConvertedComp'].mean()

,Country,ConvertedComp
0,Afghanistan,148290.125000
1,Albania,25611.000000
2,Algeria,10362.812500
3,Andorra,88640.000000
4,Angola,5292.000000
...,...,...
178,"Venezuela, Bolivarian Republic of...",6280.611111
179,Viet Nam,28342.605769
180,Yemen,36000.000000
181,Zambia,17506.400000


In [34]:
# Getting the top 5 represented countries to analyze
df[df.Country == "United States"].groupby('Gender', as_index=False)['ConvertedComp'].mean()

,Gender,ConvertedComp
0,Man,212119.882238
1,"Man;Non-binary, genderqueer, or gender non-con...",106479.933333
2,"Non-binary, genderqueer, or gender non-conforming",161044.625000
3,Woman,171652.824234
4,Woman;Man,74833.333333
5,"Woman;Man;Non-binary, genderqueer, or gender n...",154375.000000
6,"Woman;Non-binary, genderqueer, or gender non-c...",139525.925926


In [35]:
df[df.Country == "India"].groupby('Gender', as_index=False)['ConvertedComp'].mean()

,Gender,ConvertedComp
0,Man,26428.845674
1,"Man;Non-binary, genderqueer, or gender non-con...",9701.500000
2,"Non-binary, genderqueer, or gender non-conforming",502514.000000
3,Woman,18173.524194
4,Woman;Man,7921.750000
5,"Woman;Man;Non-binary, genderqueer, or gender n...",55847.000000
6,"Woman;Non-binary, genderqueer, or gender non-c...",NaN


In [36]:
df[df.Country == "United Kingdom"].groupby('Gender', as_index=False)['ConvertedComp'].mean()

,Gender,ConvertedComp
0,Man,143659.468115
1,"Man;Non-binary, genderqueer, or gender non-con...",254641.200000
2,"Non-binary, genderqueer, or gender non-conforming",112486.571429
3,Woman,108645.465753
4,Woman;Man,NaN
5,"Woman;Man;Non-binary, genderqueer, or gender n...",NaN
6,"Woman;Non-binary, genderqueer, or gender non-c...",72223.750000


In [37]:
df[df.Country == "Germany"].groupby('Gender', as_index=False)['ConvertedComp'].mean()

,Gender,ConvertedComp
0,Man,92999.737978
1,"Man;Non-binary, genderqueer, or gender non-con...",52276.428571
2,"Non-binary, genderqueer, or gender non-conforming",75398.684211
3,Woman,72474.034483
4,Woman;Man,162147.000000
5,"Woman;Man;Non-binary, genderqueer, or gender n...",NaN
6,"Woman;Non-binary, genderqueer, or gender non-c...",9804.000000


In [38]:
df[df.Country == "Canada"].groupby('Gender', as_index=False)['ConvertedComp'].mean()

,Gender,ConvertedComp
0,Man,118255.489091
1,"Man;Non-binary, genderqueer, or gender non-con...",54081.500000
2,"Non-binary, genderqueer, or gender non-conforming",63341.200000
3,Woman,97404.783333
4,Woman;Man,189078.000000
5,"Woman;Man;Non-binary, genderqueer, or gender n...",1000000.000000
6,"Woman;Non-binary, genderqueer, or gender non-c...",55832.800000


In [39]:
# Now that we have answered Question 1, we'll move on to Question 2
# Let's look at ethnicity and its contribution to salary
study_ethnicity = df['Ethnicity'].value_counts().reset_index()
study_ethnicity.head()

,index,Ethnicity
0,White or of European descent,29318
1,South Asian,4467
2,Hispanic or Latino/a/x,2256
3,Black or of African descent,1690
4,Southeast Asian,1686


In [42]:
possible_vals = ["White or of European descent","South Asian","Hispanic or Latino/a/x","Black or of African descent","Southeast Asian"] # top 5
df_ethnicity = mean_amt(df, 'Ethnicity', 'ConvertedComp', possible_vals)
df_ethnicity.sort_values('mean_col', ascending=False)

,Ethnicity,col_sum,col_total,col_squares,mean_col,var_col,std_col,lower_95,upper_95
0,White or of European descent,2.665572e+09,22290,1.595924e+15,119585.998116,5.729738e+10,239368.717171,116443.547731,122728.448500
3,Black or of African descent,8.639354e+07,901,7.291671e+13,95886.278579,7.173447e+10,267832.915072,78397.574711,113374.982448
2,Hispanic or Latino/a/x,1.791424e+08,2461,9.919339e+13,72792.523771,3.500738e+10,187102.596800,65400.215325,80184.832217
4,Southeast Asian,6.925245e+07,969,4.787483e+13,71467.957688,4.429876e+10,210472.707007,58215.697024,84720.218353
1,South Asian,1.541344e+08,2381,1.104326e+14,64735.155397,4.219013e+10,205402.361087,56484.626207,72985.684587


In [43]:
# Getting the top 5 represented countries to analyze
df[df.Country == "United States"].groupby('Ethnicity', as_index=False)['ConvertedComp'].mean()

,Ethnicity,ConvertedComp
0,Biracial,118681.818182
1,Biracial;Multiracial,98333.333333
2,Biracial;Southeast Asian,NaN
3,Black or of African descent,231346.213675
4,Black or of African descent;Biracial,759333.333333
...,...,...
116,White or of European descent;Multiracial,111916.666667
117,White or of European descent;Multiracial;South...,780000.000000
118,White or of European descent;South Asian,113062.500000
119,White or of European descent;South Asian;Multi...,733666.666667


In [46]:
df[df.Country == "India"].groupby('Ethnicity', as_index=False)['ConvertedComp'].mean()

,Ethnicity,ConvertedComp
0,Biracial,NaN
1,Biracial;Southeast Asian,13962.000000
2,Black or of African descent,8628.000000
3,Black or of African descent;East Asian;Hispani...,NaN
4,Black or of African descent;East Asian;Hispani...,NaN
5,Black or of African descent;East Asian;Hispani...,519546.500000
6,Black or of African descent;East Asian;Hispani...,55847.000000
7,Black or of African descent;East Asian;Hispani...,NaN
8,Black or of African descent;East Asian;White o...,NaN
9,Black or of African descent;Middle Eastern;Ind...,NaN


In [47]:
df[df.Country == "United Kingdom"].groupby('Ethnicity', as_index=False)['ConvertedComp'].mean()

,Ethnicity,ConvertedComp
0,Biracial,180427.545455
1,Black or of African descent,195106.757576
2,Black or of African descent;Biracial,116333.000000
3,Black or of African descent;White or of Europe...,373559.000000
4,Black or of African descent;White or of Europe...,74363.000000
5,Black or of African descent;White or of Europe...,NaN
6,Black or of African descent;White or of Europe...,359989.000000
7,East Asian,163345.687500
8,East Asian;Southeast Asian,89189.000000
9,East Asian;White or of European descent,NaN


In [48]:
df[df.Country == "Germany"].groupby('Ethnicity', as_index=False)['ConvertedComp'].mean()

,Ethnicity,ConvertedComp
0,Biracial,95847.333333
1,Black or of African descent,140532.636364
2,Black or of African descent;Biracial,54049.000000
3,Black or of African descent;East Asian;Hispani...,71345.000000
4,Black or of African descent;East Asian;Hispani...,NaN
5,Black or of African descent;Hispanic or Latino...,63345.666667
6,Black or of African descent;Hispanic or Latino...,NaN
7,Black or of African descent;Hispanic or Latino...,44320.000000
8,Black or of African descent;Hispanic or Latino...,97288.000000
9,Black or of African descent;Middle Eastern,59454.000000


In [49]:
df[df.Country == "Canada"].groupby('Ethnicity', as_index=False)['ConvertedComp'].mean()

,Ethnicity,ConvertedComp
0,Biracial,93214.500000
1,Biracial;Multiracial,NaN
2,Black or of African descent,130423.307692
3,Black or of African descent;Biracial,108000.000000
4,Black or of African descent;East Asian;Hispani...,NaN
5,Black or of African descent;East Asian;Hispani...,1000000.000000
6,Black or of African descent;East Asian;Hispani...,55211.000000
7,Black or of African descent;Hispanic or Latino...,NaN
8,Black or of African descent;Hispanic or Latino...,61640.000000
9,Black or of African descent;Middle Eastern,105884.000000


In [52]:
# Now on to Question 3:
study_higher_ed = df['EdLevel'].value_counts().reset_index()
study_higher_ed.head()

,index,EdLevel
0,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",26542
1,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",13112
2,Some college/university study without earning ...,7239
3,"Secondary school (e.g. American high school, G...",4771
4,"Associate degree (A.A., A.S., etc.)",1843


In [54]:
# Average Job Satisfaction
study_job_sat = df['JobSat'].value_counts().reset_index()
study_job_sat.head()

,index,JobSat
0,Very satisfied,14611
1,Slightly satisfied,13930
2,Slightly dissatisfied,7124
3,Neither satisfied nor dissatisfied,5783
4,Very dissatisfied,3746


In [3]:
df.groupby('EdLevel', as_index=False)['ConvertedComp'].mean()

,EdLevel,ConvertedComp
0,"Associate degree (A.A., A.S., etc.)",106815.832192
1,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",107638.145312
2,I never completed any formal education,104351.616883
3,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",103104.775496
4,"Other doctoral degree (Ph.D., Ed.D., etc.)",122849.059545
5,Primary/elementary school,114752.679739
6,"Professional degree (JD, MD, etc.)",72062.004193
7,"Secondary school (e.g. American high school, G...",79067.561392
8,Some college/university study without earning ...,97895.641703


In [7]:
df[df.EdLevel == "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,3779
1,Slightly satisfied,3539
2,Slightly dissatisfied,1778
3,Neither satisfied nor dissatisfied,1433
4,Very dissatisfied,941


In [9]:
df[df.EdLevel == "Other doctoral degree (Ph.D., Ed.D., etc.)"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,651
1,Slightly satisfied,415
2,Slightly dissatisfied,163
3,Very dissatisfied,134
4,Neither satisfied nor dissatisfied,120


In [10]:
df[df.EdLevel == "Professional degree (JD, MD, etc.)"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,189
1,Slightly satisfied,189
2,Slightly dissatisfied,124
3,Neither satisfied nor dissatisfied,72
4,Very dissatisfied,66


In [15]:
# Finally, Question 4:
pd.set_option('display.max_colwidth', None)
study_overtime = df['NEWOvertime'].value_counts().reset_index()
study_overtime.head()

,index,NEWOvertime
0,Sometimes: 1-2 days per month but less than weekly,11547
1,Often: 1-2 days per week or more,11027
2,Occasionally: 1-2 days per quarter but less than monthly,9468
3,Rarely: 1-2 days per year or less,6471
4,Never,4718


In [13]:
df[df.NEWOvertime == "Never"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,1420
1,Slightly satisfied,1386
2,Slightly dissatisfied,763
3,Neither satisfied nor dissatisfied,624
4,Very dissatisfied,497


In [14]:
df[df.NEWOvertime == "Rarely: 1-2 days per year or less"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,2044
1,Slightly satisfied,1962
2,Slightly dissatisfied,1070
3,Neither satisfied nor dissatisfied,855
4,Very dissatisfied,517


In [16]:
df[df.NEWOvertime == "Occasionally: 1-2 days per quarter but less than monthly"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,3093
1,Slightly satisfied,3023
2,Slightly dissatisfied,1439
3,Neither satisfied nor dissatisfied,1212
4,Very dissatisfied,683


In [17]:
df[df.NEWOvertime == "Sometimes: 1-2 days per month but less than weekly"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,3839
1,Slightly satisfied,3735
2,Slightly dissatisfied,1726
3,Neither satisfied nor dissatisfied,1401
4,Very dissatisfied,824


In [18]:
df[df.NEWOvertime == "Often: 1-2 days per week or more"]['JobSat'].value_counts().reset_index()

,index,JobSat
0,Very satisfied,3546
1,Slightly satisfied,3208
2,Slightly dissatisfied,1869
3,Neither satisfied nor dissatisfied,1361
4,Very dissatisfied,1011
